In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchinfo import summary         
import numpy as np
import torchvision.transforms as tf
import transformers                    
from tensorboardX import SummaryWriter 
from pkg_resources import packaging    
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from openTSNE import TSNE              
import pandas as pd
import seaborn as sns

from tqdm import tqdm
import random
import re
import time
import math

In [ ]:
model = torch.load('paras.txt')

In [ ]:
def get_images_by_category(dataset, category):
    category_indices = [i for i, label in enumerate(dataset.targets) if label == category]
    images = [dataset[i][0] for i in category_indices]
    return images

# Revert the transformations in reverse order
reverted_tf = tf.Compose([
    tf.Normalize((-0.485 / 0.229, -0.456 / 0.224, -0.406 / 0.225), (1 / 0.229, 1 / 0.224, 1 / 0.225)),
    tf.Normalize((0, 0, 0), (1, 1, 1)),  # Unnormalize the tensor
    tf.ConvertImageDtype(torch.uint8),
    tf.ToPILImage(),
])

In [ ]:
dog_images = get_images_by_category(test_set, 5)  # "dog" category label is 5

plt.figure(figsize=(10, 5))
for i in range(2):
    plt.subplot(1, 2, i + 1)
    plt.imshow(reverted_tf(dog_images[i]))
    plt.axis('off')
    plt.title('Dog')
plt.show()

direct

In [ ]:
def NEB(image1, image2, target_index, intermediate_no=10, model=model, mode='standard', k=0.5, max_itr=1000):
    for param in model.parameters():
        param.requires_grad = False

    # fc_layer = model.fc
    # model_without_fc = torch.nn.Sequential(*list(model.children())[:-1])

    image1 = image1.to(device)
    image1_batch = image1.unsqueeze(0)
    output1 = model(image1_batch)
    image2 = image2.to(device)
    image2_batch = image2.unsqueeze(0)
    output2 = model(image2_batch)
    spacing = (image2_batch - image1_batch) / (intermediate_no + 1)
    intermediate_images = torch.stack([image1_batch + spacing * i for i in range(1, intermediate_no + 1)])

    for _ in range(max_itr):
        f_total = 0
        inputs, outputs, potential_grad = [], [], []
        for i in range(intermediate_no):
            inputs.append(torch.tensor(intermediate_images[i].clone(), requires_grad=True))
            outputs.append(model(intermediate_images))
            outputs[-1].backward()
            with torch.no_grad():
                potential_grad.append(inputs[-1].grad.copy())
        direction = get_direction(inputs, outputs, target_index)
        for i in range(intermediate_no):
            f_potential = -potential_grad[i] + potential_grad @ direction[i] * direction[i]
            f_spring = k * (torch.abs(inputs[i+1] - inputs[i]) - torch.abs(inputs[i] - inputs[i-1])) * direction[i]
            f = f_potential + f_spring
            f_total += f
            with torch.no_grad():
                inputs[i] += lr * f
        if f_total < 0.01:
            break

    return intermediate_images

def get_direction(inputs, outputs, target_index):
    pass

latent space

In [ ]:
def NEB(image1, image2, target_index, intermediate_no=10, model=model, mode='standard', k=0.5, max_itr=1000):
    for param in model.parameters():
        param.requires_grad = False

    fc_layer = model.fc
    model_without_fc = torch.nn.Sequential(*list(model.children())[:-1])

    image1 = image1.to(device)
    image1_batch = image1.unsqueeze(0)
    output1 = model_without_fc(image1_batch)
    image2 = image2.to(device)
    image2_batch = image2.unsqueeze(0)
    output2 = model_without_fc(image2_batch)
    spacing = (output2 - output1) / (intermediate_no + 1)
    intermediate_images = torch.stack([output1 + spacing * i for i in range(1, intermediate_no + 1)])

    intermediate_images = NEB_train(intermediate_images, fc_layer, target_index, intermediate_no, k=k, max_itr=max_itr)

    return intermediate_images

def NEB_train(intermediate_images, fc_layer, target_index, intermediate_no, k, max_itr, tol):

    for _ in range(max_itr):
        f_total = 0
        inputs, outputs, potential_grad = [], [], []
        for i in range(intermediate_no):
            inputs.append(torch.tensor(intermediate_images[i].clone(), requires_grad=True))
            outputs.append(fc_layer(intermediate_images))
            outputs[-1].backward()
            with torch.no_grad():
                potential_grad.append(inputs[-1].grad.copy())
        direction = get_direction(inputs, outputs, target_index)
        for i in range(intermediate_no):
            f_potential = -potential_grad[i] + potential_grad @ direction[i] * direction[i]
            f_spring = k * (torch.abs(inputs[i+1] - inputs[i]) - torch.abs(inputs[i] - inputs[i-1])) * direction[i]
            f = f_potential + f_spring
            f_total += f
            with torch.no_grad():
                inputs[i] += lr * f
        if f_total < tol:
            break

def get_direction(inputs, outputs, target_index):
    pass

In [ ]:
indices = np.random.randint(0, 1000, size=2)
image1, image2 = dog_images[indices[0]], dog_images[indices[1]]

In [ ]:
intermediate_images = NEB(image1, image2, intermediate_no=10)

In [ ]:
import torch
import torch.nn.functional as F

def blur_layer(x, kernel_size=3, sigma=1.0):
    padding = kernel_size // 2
    channels = x.size(1)

    kernel = torch.tensor([[1, 2, 1],
                           [2, 4, 2],
                           [1, 2, 1]], dtype=torch.float32) / 16.0  # Gaussian blur kernel

    kernel = kernel.view(1, 1, kernel_size, kernel_size)
    kernel = kernel.expand(channels, -1, -1, -1).to(x.device)

    blurred = F.conv2d(x, kernel, padding=padding, groups=channels)

    return blurred

In [ ]:
indices = np.random.randint(0, 1000, size=2)
image1 = dog_images[indices[0]]

image_size = (3, 224, 224)  # Size of the image tensor
image2 = torch.randn(image_size)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
image2 = normalize(image2)

for param in model.parameters():
    param.requires_grad = False

import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.optim import Adam
import torch.nn.functional as F

# Load the pre-trained ResNet model and extract the model without the FC layer
model_without_fc = torch.nn.Sequential(*list(model.children())[:-1])
transform_gaussion = transforms.GaussianBlur(21,10)

image1 = image1.to(device)
image2 = image2.to(device)
image1 = image1.unsqueeze(0)  # Assuming image1 is your input image tensor
image2 = image2.unsqueeze(0)  # Assuming image2 is your target image tensor

# Create a tensor to hold the optimized image1
image1_optimized = torch.tensor(image2.clone(), requires_grad=True)

# Set up the optimizer
# optimizer = Adam([image1_optimized], lr=0.01)

# Optimization loop
num_iterations = 1000
losses = []
for _ in range(num_iterations):
    # Forward pass with image1_optimized

    image1_optimized = torch.tensor(transform_gaussion(image1_optimized).clone(), requires_grad=True)
    optimizer = Adam([image1_optimized], lr=0.01)
    output1 = model_without_fc(image1_optimized)

    # Forward pass with image2
    output2 = model_without_fc(image1)

    # Compute the loss between output1 and output2
    loss = torch.mean((output1 - output2) ** 2)
    losses.append(loss)

    # Zero out the gradients
    optimizer.zero_grad()

    # Compute gradients of the loss with respect to image1_optimized
    loss.backward()

    # Update image1_optimized
    optimizer.step()
    if loss < 0.01:
        break

# Access the optimized image1
optimized_image1 = image1_optimized.squeeze()
print(losses[0], losses[-1])


In [ ]:
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(reverted_tf(image1.squeeze()))
plt.axis('off')
plt.title('Dog')

plt.subplot(1, 2, 2)
plt.imshow(reverted_tf(optimized_image1))
plt.axis('off')
plt.title('Dog')

plt.show()

In [ ]:
plt.plot([tensor.cpu().detach().numpy() for tensor in losses])